code borrowed from:
https://github.com/huggingface/notebooks/blob/main/examples/token_classification.ipynb

In [1]:
LOCAL = False   # training on local Mac vs. in Colab

import os
import sys
import time

if LOCAL:
    USE_MPS_DEVICE = True   # for training on M1 chip
    BASE_PATH = "/Users/carolanderson/Dropbox/"

else:
    USE_MPS_DEVICE = False
    BASE_PATH = "/content/drive/My Drive/"
    ! pip install comet-ml
    ! pip install datasets
    ! pip install evaluate
    ! pip install transformers[torch]
    ! pip install seqeval
    from google.colab import drive
    drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.1/510.1 kB 37.4 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.0
    Uninstalling websocket-client-1.6.0:
      Successfully uninstalled websocket-client-1.6.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.8 MB/s eta 0:00:00
  

In [2]:
import comet_ml
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import evaluate
import numpy as np
import transformers
from transformers import AutoTokenizer, \
                        AutoConfig, \
                        DataCollatorForTokenClassification, \
                        AutoModelForTokenClassification, \
                        TrainingArguments, \
                        Trainer, \
                        EarlyStoppingCallback

In [3]:
comet_ml.init(project_name='food-ner')

In [4]:
def get_current_time():
    t = time.localtime()
    current_time = time.strftime("%H_%M_%S", t)
    return current_time


def read_conll_file(file):
    """
    Given a file in CoNLL format, read in tokens and labels. Treat each sentence as a training example.
    :param file: file in CoNLL format; tokens are assumed to be in the first column and labels in the last column.
    :returns a nested list, in which each sublist is a sentence and contains a sublist [token, label] for each token.

    .. note:: Ignores document boundaries and treats each sentence as an independent training example.
    """
    documents = []  # holds all documents
    sentence = [] # will hold the first sentence
    with open(file, 'r') as infile:
        for line in infile:
            if '-DOCSTART-' in line:  # beginning of a new document; ignore this since we will treat each sentence as a training example
                continue
            elif not line.split():  # beginning of a new sentence
                if sentence:
                    documents.append(sentence)
                sentence = []
            else:
                token, *other_columns, label = line.split()
                sentence.append([token, label])
    return documents


def convert_to_dataset(data, label_map):
    formatted_data = {"tokens": [], "ner_tags": []}
    for sentence in data:
        tokens = [token_data[0] for token_data in sentence]
        ner_tags = [label_map[token_data[1]] for token_data in sentence]
        formatted_data["tokens"].append(tokens)
        formatted_data["ner_tags"].append(ner_tags)
    return Dataset.from_dict(formatted_data)


def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


def compute_metrics(p):
    experiment = comet_ml.get_global_experiment()
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    result_dict = {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
    experiment.log_metrics(result_dict)
    return result_dict

# Load datasets

In [5]:
train_data = read_conll_file(os.path.join(BASE_PATH, "nlp_data", "recipe_data", "20200523_food_gold_train.conll"))
validation_data = read_conll_file(os.path.join(BASE_PATH, "nlp_data",  "recipe_data", "20200523_food_gold_dev.conll"))
test_data = read_conll_file(os.path.join(BASE_PATH, "nlp_data",  "recipe_data", "20200523_food_gold_test.conll"))

In [6]:
label_list = sorted(list(set([token_data[1] for sentence in train_data for token_data in sentence])))
label_map = {label: i for i, label in enumerate(label_list)}

# mappings for config
label2id = label_map
id2label = {v:k for k, v in label2id.items()}

train_dataset = convert_to_dataset(train_data, label_map)
validation_dataset = convert_to_dataset(validation_data, label_map)
test_dataset = convert_to_dataset(test_data, label_map)

food_datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset,
})

# Tokenize and align labels to tokens

In [8]:
model_checkpoint = "roberta-base"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

In [13]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [14]:
tokenized_datasets = food_datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/592 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/194 [00:00<?, ? examples/s]

# Set up training

In [33]:
task = "ner"
batch_size = 4
experiment_id = "20230705_" + get_current_time()

In [34]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir = os.path.join(BASE_PATH, "food_ner_models", f"{experiment_id}-{model_name}-finetuned-{task}"),
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps = 20,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
    use_mps_device=USE_MPS_DEVICE,
    load_best_model_at_end=True,
    metric_for_best_model = "f1"
)

In [35]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [36]:
config = AutoConfig.from_pretrained(model_checkpoint, label2id=label2id, id2label=id2label)

In [37]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, config=config)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [38]:
metric = evaluate.load("seqeval")

In [39]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(2)]
)

# Train

In [40]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/carolmanderson/food-ner/81e29a1194d94011bbe50af0d342818c



Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.073300,0.043431,0.926050,0.937872,0.931924,0.986352
2,0.031500,0.036146,0.946565,0.949787,0.948173,0.989191
3,0.023900,0.037944,0.938912,0.954894,0.946835,0.988825
4,0.013500,0.038335,0.955707,0.954894,0.955300,0.990474
5,0.020600,0.038603,0.953586,0.961702,0.957627,0.990840
6,0.009500,0.047093,0.956110,0.945532,0.950792,0.990382
7,0.007800,0.044359,0.949410,0.958298,0.953833,0.990748


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/carolmanderson/food-ner/81e29a1194d94011bbe50af0d342818c
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [7]                   : (0.986351561784373, 0.9908399743519282)
COMET INFO:     epoch [59]                     : (0.14, 7.0)
COMET INFO:     eval/accuracy [7]              : (0.986351561784373, 0.9908399743519282)
COMET INFO:     eval/f1 [7]                    : (0.9319238900634249, 0.9576271186440678)
COMET INFO:     eval/loss [7]                  : (0.03614619001746178, 0.047092877328395844)
COMET INFO:     eval/precision [7]             : (0.9260504201680673, 0.9561101549053356)
COMET INFO:     eval/reca

TrainOutput(global_step=1036, training_loss=0.03952890948093996, metrics={'train_runtime': 374.5643, 'train_samples_per_second': 15.805, 'train_steps_per_second': 3.951, 'total_flos': 634466923574400.0, 'train_loss': 0.03952890948093996, 'epoch': 7.0})